In [ ]:
# Need help identifying the environment from your current notebook?
#!python -m ensurepip --default-pip
#!python -m pip --version
import sys
sys.executable
!which python3
!which pip
!pip -V

In [ ]:
# pip install google-adk
# This guarantees installation into the exact kernel environment.

# !{sys.executable} -m pip install google-adk

# !{sys.executable} -m pip install google-adk --break-system-packages

In [ ]:
# !pip install python-dotenv

# This guarantees installation into the exact kernel environment.
# !{sys.executable} -m pip install python-dotenv

In [ ]:
# !pip install yfinance --upgrade
# !python3 -m pip install yfinance --upgrade

In [ ]:
# !pip install feedparser

In [36]:
from dotenv import load_dotenv
import os

try:
    load_dotenv()
    api_key = os.getenv("GOOGLE_API_KEY")
    if api_key is None:
        raise Exception("GOOGLE_API_KEY is missing")
    print("✅ Gemini API key setup complete.")
    # print(api_key)
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")


✅ Gemini API key setup complete.


In [37]:
from typing import Any, Dict
from google.adk.agents import Agent, SequentialAgent, LlmAgent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner, InMemoryRunner
from google.adk.tools import google_search
from google.genai import types
from google.adk.models.google_llm import Gemini

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [ ]:
# First ADK Agent
root_agent = Agent(name="helpful_assistant", model="gemini-2.5-flash-lite", 
                   description="a simple assistant that can answer questions",
                   instruction="you are a helpful assistant. Use Google search for current info or if unsure.",
                   tools=[google_search])
print("Root Agent defined.")

In [ ]:
# Run your agent
runner = InMemoryRunner(agent=root_agent)
print("runner created.")

In [ ]:
# run in debug mode for prototyping
response = await runner.run_debug("What is Capstone project in Kaggle?")

In [ ]:
# multi agents system
# expense_analyzer
expense_analyzer_agent = Agent(name = "expanse_analyzer_assistant", model = "gemini-2.5-flash-lite",
                              description = "assitant which do analysis of expenses of a person",
                              instruction = "you are a smart anaylsis assistant. You analyze the expenses done by a person in a month.",
                              tools=[google_search],
                              output_key="result")
print("expense_analyzer_agent created.")

In [ ]:
# financial_advisor
financial_advisor_agent = Agent(name = "financial_advisor_assistant", model = "gemini-2.5-flash-lite",
                              description = "assitant who advisor on investment, savings & loan decision to a person",
                              instruction = "you are a smart advisor assistant. On the basis of result of expanse_analyzer_assistant, suggest good financial advise to a person.",
                              tools=[google_search],
                              output_key="advise")
print("financial_advisor_agent created.")

In [ ]:
# root agent - orchestrator
fin_track_agent = SequentialAgent(name = "finTrack", 
                                 sub_agents=[expense_analyzer_agent, financial_advisor_agent])

# Similarly you can create ParallelAgents

print("SequentialAgent fin_track_agent is created.")

In [ ]:
query = "Rahul got salary of 20,000/- today. He has pending bills on his credit cards of around 5,000/- but he decided to buy his favourite video game worth 18,000/-. What should Rahul do?"
print(query)

In [ ]:
runner = InMemoryRunner(agent=fin_track_agent)
response = await runner.run_debug(query)

In [ ]:
# Custom Function Tools -> you own code, solves domain specifi problems, your control over functionality
def risk_assesment(expense_type: str) -> str :
    """Take the expense type and decide the risk invloved with the expense.
    Args: 
        expense_type: String which describes the type of expense like rent, loan, bills, etc.
    Returns:
        String with the resultant risk as evaluated by the internal logic
    """
    if expense_type == "rent":
        return "NECESSITY"
    else:
        return "NON_NECESSITY"

In [ ]:
print(risk_assesment("rent"))

In [ ]:
risk_assesser_agent = LlmAgent(name = "risk_assesment_agent",
                              model = Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
                              instruction = "You are a smart risk assesment assistant, who identifies the risk from the given expenses of a person",
                              tools = [risk_assesment])
print("risk_assesser_agent is created")

In [ ]:
query = "I have to pay 10,000/- rent from 20,000/- salary. what is the risk level of this expense?"
print(query)

In [ ]:
risk_assesser_runner = InMemoryRunner(agent=risk_assesser_agent)
_ = await risk_assesser_runner.run_debug(query)

In [ ]:
# Agent Tool -> Agent act as a tool, ex. calcAgent which generated python code to do calculations

# Agents vs Sub-agents
# Agents => Delegation for specific tasks (like calculations)
# Sub Agents => Handoff to specialists (like customer support agent)

In [ ]:
# Create a MCP server and call it here
# Idea - MCP for news related to finance & market

In [ ]:
import feedparser
# can be used as tool functions
def fetch_yahoo_finance_news(limit=10):
    url = "https://finance.yahoo.com/rss/topstories"
    
    feed = feedparser.parse(url)
    
    articles = []
    for entry in feed.entries[:limit]:
        articles.append({
            "title": entry.title,
            "link": entry.link,
            "published": entry.published
        })
    
    return articles


# ▶️ Fetch and print
articles = fetch_yahoo_finance_news(limit=10)

for idx, a in enumerate(articles, start=1):
    print(f"\n{idx}. {a['title']}")
    print(f"   Link: {a['link']}")
    print(f"   Published: {a['published']}")


In [ ]:
# Session Management
# LLMs are inherently stateless.
# Sessions -> Short term memory management
# Memory -> Long term memory

# Sessions: 
#     - single, continous conversations
#     - single user & agent, not shared with othr user
#     - session history for an Agent is not shared with other Agents
# Sessions 
#     1. Events - building blocks of conversation
#     2. State - Agents's scratchpad, global {key, value} pair storage

# Session Manager - SessionService - storage layer
# Runner - orchestation layer

# Session = A notebook 📓
# Events = Individual entries in a single page 📝
# SessionService = The filing cabinet storing notebooks 🗄️
# Runner = The assistant managing the conversation 🤖


In [ ]:
# a user logged in app
# starts its query
APP_NAME = "capstone"
USER_ID = "prasri"
SESSION = "Monday_17nov"

MODEL_NAME = "gemini-2.5-flash-lite"

root_agent = Agent(model =Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
                   name="fin_trac_agent", description="A smart finance tracker assistant")

session_service = InMemorySessionService()

runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)
print("stateful agent created!")

In [ ]:
# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

In [ ]:
# test sessions
await run_session(runner, 
                  [
                      "Hi, My name is Prakhar and I spent 15000 in shopping.",
                      "Who am I and how much I spent?"
                  ],
                  "stateful-agentic-session")

In [38]:
import sqlite3
from datetime import datetime
import os

DB_FILE = "fintrack.db"

def initialize_database():
    """Create all tables if they don't exist"""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    # Users table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            user_id TEXT PRIMARY KEY,
            name TEXT NOT NULL,
            email TEXT,
            current_balance REAL DEFAULT 0,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    # Transactions table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS transactions (
            transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id TEXT NOT NULL,
            amount REAL NOT NULL,
            category TEXT,
            transaction_type TEXT NOT NULL,
            description TEXT,
            date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (user_id) REFERENCES users(user_id)
        )
    ''')
    
    # Bills table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS bills (
            bill_id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id TEXT NOT NULL,
            bill_name TEXT NOT NULL,
            amount REAL NOT NULL,
            due_date DATE NOT NULL,
            status TEXT DEFAULT 'pending',
            penalty_rate REAL DEFAULT 18.0,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (user_id) REFERENCES users(user_id)
        )
    ''')
    
    # Savings plans table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS savings_plans (
            plan_id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id TEXT NOT NULL,
            item_name TEXT NOT NULL,
            target_amount REAL NOT NULL,
            monthly_save_amount REAL,
            monthly_invest_amount REAL,
            target_months INTEGER,
            status TEXT DEFAULT 'active',
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (user_id) REFERENCES users(user_id)
        )
    ''')
    
    # Monthly reports table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS monthly_reports (
            report_id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id TEXT NOT NULL,
            month TEXT NOT NULL,
            starting_balance REAL,
            ending_balance REAL,
            total_income REAL DEFAULT 0,
            total_expenses REAL DEFAULT 0,
            total_savings REAL DEFAULT 0,
            profitability REAL DEFAULT 0,
            interventions_count INTEGER DEFAULT 0,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (user_id) REFERENCES users(user_id)
        )
    ''')
    
    # Create indexes for performance
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_transactions_user ON transactions(user_id)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_transactions_date ON transactions(date)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_bills_user ON bills(user_id)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_savings_user ON savings_plans(user_id)')
    
    conn.commit()
    conn.close()
    print("✓ Database initialized successfully!")

def get_db_connection():
    """Get a database connection"""
    conn = sqlite3.connect(DB_FILE)
    conn.row_factory = sqlite3.Row  # Access columns by name
    return conn

def create_demo_user():
    """Create a demo user with sample data for testing"""
    conn = get_db_connection()
    cursor = conn.cursor()
    
    # Check if demo user exists
    cursor.execute("SELECT user_id FROM users WHERE user_id = 'demo_user'")
    if cursor.fetchone():
        print("Demo user already exists")
        conn.close()
        return
    
    # Create demo user
    cursor.execute('''
        INSERT INTO users (user_id, name, email, current_balance)
        VALUES (?, ?, ?, ?)
    ''', ('demo_user', 'Demo User', 'demo@fintrack.com', 20000.0))
    
    # Add some pending bills
    cursor.execute('''
        INSERT INTO bills (user_id, bill_name, amount, due_date, status, penalty_rate)
        VALUES (?, ?, ?, date('now', '+5 days'), 'pending', 20.0)
    ''', ('demo_user', 'Credit Card Bill', 5000.0))
    
    cursor.execute('''
        INSERT INTO bills (user_id, bill_name, amount, due_date, status, penalty_rate)
        VALUES (?, ?, ?, date('now', '+10 days'), 'pending', 18.0)
    ''', ('demo_user', 'Electricity Bill', 1500.0))
    
    # Add some past transactions
    cursor.execute('''
        INSERT INTO transactions (user_id, amount, category, transaction_type, description, date)
        VALUES (?, ?, ?, ?, ?, datetime('now', '-15 days'))
    ''', ('demo_user', 25000.0, 'necessary', 'income', 'Monthly Salary'))
    
    cursor.execute('''
        INSERT INTO transactions (user_id, amount, category, transaction_type, description, date)
        VALUES (?, ?, ?, ?, ?, datetime('now', '-10 days'))
    ''', ('demo_user', 3000.0, 'necessary', 'expense', 'Grocery Shopping'))
    
    cursor.execute('''
        INSERT INTO transactions (user_id, amount, category, transaction_type, description, date)
        VALUES (?, ?, ?, ?, ?, datetime('now', '-5 days'))
    ''', ('demo_user', 2000.0, 'useless', 'expense', 'Impulse online shopping'))
    
    conn.commit()
    conn.close()
    print("✓ Demo user created with sample data!")

if __name__ == "__main__":
    # Run this script to set up the database
    print("Initializing FinTrack Database...")
    initialize_database()
    create_demo_user()
    print("\n✓ Database setup complete!")
    print(f"✓ Database file: {os.path.abspath(DB_FILE)}")

Initializing FinTrack Database...
✓ Database initialized successfully!
✓ Demo user created with sample data!

✓ Database setup complete!
✓ Database file: /Users/p0s07cu/Desktop/agenticAI/fintrack.db


In [40]:
# from database import get_db_connection

def test_database():
    conn = get_db_connection()
    cursor = conn.cursor()
    
    # Test: Get demo user
    cursor.execute("SELECT * FROM users WHERE user_id = 'demo_user'")
    user = cursor.fetchone()
    print(f"User: {user['name']}, Balance: ₹{user['current_balance']}")
    
    # Test: Get pending bills
    cursor.execute("SELECT * FROM bills WHERE user_id = 'demo_user' AND status = 'pending'")
    bills = cursor.fetchall()
    print(f"\nPending Bills: {len(bills)}")
    for bill in bills:
        print(f"  - {bill['bill_name']}: ₹{bill['amount']}")
    
    # Test: Get transactions
    cursor.execute("SELECT * FROM transactions WHERE user_id = 'demo_user'")
    transactions = cursor.fetchall()
    print(f"\nTransactions: {len(transactions)}")
    for txn in transactions:
        print(f"  - {txn['description']}: ₹{txn['amount']} ({txn['transaction_type']})")
    
    conn.close()

if __name__ == "__main__":
    test_database()

User: Demo User, Balance: ₹20000.0

Pending Bills: 2
  - Credit Card Bill: ₹5000.0
  - Electricity Bill: ₹1500.0

Transactions: 3
  - Monthly Salary: ₹25000.0 (income)
  - Grocery Shopping: ₹3000.0 (expense)
  - Impulse online shopping: ₹2000.0 (expense)


In [41]:
# from database import get_db_connection

def get_user_balance(user_id: str) -> dict:
    """
    Get user's current balance and pending bills.
    This tool helps the agent understand user's financial situation.
    """
    conn = get_db_connection()
    cursor = conn.cursor()
    
    # Get user balance
    cursor.execute("SELECT current_balance FROM users WHERE user_id = ?", (user_id,))
    user = cursor.fetchone()
    
    if not user:
        return {"error": "User not found"}
    
    current_balance = user['current_balance']
    
    # Get total pending bills
    cursor.execute("""
        SELECT SUM(amount) as total_bills 
        FROM bills 
        WHERE user_id = ? AND status = 'pending'
    """, (user_id,))
    
    bills_result = cursor.fetchone()
    pending_bills = bills_result['total_bills'] if bills_result['total_bills'] else 0
    
    conn.close()
    
    # Calculate available cash after bills
    available_cash = current_balance - pending_bills
    
    return {
        "current_balance": current_balance,
        "pending_bills_total": pending_bills,
        "available_cash": available_cash,
        "status": "healthy" if available_cash > 0 else "warning"
    }


# Test function
if __name__ == "__main__":
    result = get_user_balance("demo_user")
    print("User Balance Info:")
    print(f"  Current Balance: ₹{result['current_balance']}")
    print(f"  Pending Bills: ₹{result['pending_bills_total']}")
    print(f"  Available Cash: ₹{result['available_cash']}")
    print(f"  Status: {result['status']}")

User Balance Info:
  Current Balance: ₹20000.0
  Pending Bills: ₹6500.0
  Available Cash: ₹13500.0
  Status: healthy
